In [ ]:
import requests
import pandas as pd
import geopandas as gp
import numpy as np
from pathlib import Path

### Helpers

In [ ]:
outdir = Path('../../../../data/working/')

In [ ]:
def safelyget(alist, idx, default='N/A'):
    """Returns alist[idx] if exists, else default"""
    try:
        return alist[idx]
    except (KeyError, IndexError):
        return default
    
def pick_icao(sidslist):
    """Returns the ICAO code from a station ID list from ACIS"""
    try:
        return [item.split()[0] for item in sidslist if item.endswith(' 5')][0]
    except (AttributeError, IndexError):
        # There's no ICAO code in the list
        return ''

### Get all ACIS stations that have avg daily temp data, with their data validity rage, ID and lat/lon

Variable 13 = snow water equivalent

In [ ]:
baseurl = 'http://data.rcc-acis.org/StnMeta'
params = {
    'state': "AK",
    'meta': "name,uid,sids,ll,elev,valid_daterange",
    'elems': "avgt,snow"
}

In [ ]:
resp = requests.get(url=baseurl, params=params)

In [ ]:
stationdat = resp.json()['meta']
len(stationdat)

In [ ]:
stationdat

In [ ]:
records = [   
    dict(
        name=stat['name'],
        acisID=stat['uid'],
        icao=pick_icao(stat['sids']),
        longitude=safelyget(safelyget(stat, 'll', []), 0, np.nan),
        latitude=safelyget(safelyget(stat, 'll', []), 1, np.nan),
        elev_ft=safelyget(stat, 'elev', np.nan), 
        valid_avgT_start=safelyget(stat['valid_daterange'][0], 0, ''),
        valid_avgT_end=safelyget(stat['valid_daterange'][0], 1, ''),
        valid_snowdepth_start=safelyget(stat['valid_daterange'][1], 0, ''),
        valid_snowdepth_end=safelyget(stat['valid_daterange'][1], 1, ''),
    )
    for stat in stationdat
]


In [ ]:
stationDF = pd.DataFrame.from_records(records)
stationDF

Let's drop stations with no valid temperature data as well as too short / old temp series

In [ ]:
stationDF = stationDF[~(stationDF.valid_avgT_start=='')] 
stationDF = stationDF[stationDF.valid_avgT_end > '1990']
stationDF = stationDF.sort_values(by='name')
stationDF = stationDF.reset_index(drop=True)
stationDF

In [ ]:
stationDF.to_csv(outdir / "ACIS_stations_AK_cleaned.csv")

In [ ]:
selectedstationsDF = stationDF[
    (stationDF.valid_avgT_start < '1980-01-01') & (stationDF.valid_avgT_end > '2023-02-01')]
selectedstationsDF

In [ ]:
selectedstationsDF.to_csv(outdir / "ACIS_stations_AK_recentTemps.csv")

In [ ]:
stationsGDF = gp.GeoDataFrame(
    stationDF, geometry=gp.points_from_xy(stationDF.longitude, stationDF.latitude))
stationsGDF = stationsGDF.set_crs('epsg:4326')
stationsGDF

In [ ]:
stationsGDF

In [ ]:
stationsGDF.to_file(outdir / "ACISAK_stations.gpkg", driver='GPKG')